In [127]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pandasql as ps
import string
import random
from faker import Faker

from lib.masking_functions import mask, bucketize_age, blur_zip, generalize_diagnosis, add_relative_noise, blur_phone

In [128]:
def plot_probabilities(dist):
    dist_copy = dist.copy().sort_index()
    # labels = dist.index.to_series().apply(lambda x: '{0}-{1}'.format(*x))
    # labels = dist.index

    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(range(len(dist_copy.values)),dist_copy.values)
    plt.show()

## Data Generation

In [129]:
seed = 0

fake = Faker('de_DE')

fake.seed_instance(seed)
np.random.seed(seed)
random.seed(seed)

from faker.providers import BaseProvider

# create new provider class for diagnosis following the ICD_10_GM medical code
class ICD_10_GM_Provider(BaseProvider):
    def diagnosis(self) -> str:
        letter = np.random.choice(string.ascii_uppercase.replace("U", "").replace("W", "").replace("X", ""))
        number = np.trunc(np.random.uniform(0.0, 99.9)*10)/(10)
        if number < 10:
            return letter+"0"+str(number)
        else:
            return letter+str(number)
        
fake.add_provider(ICD_10_GM_Provider)

In [130]:
nRows = 4000

pid = np.arange(1,nRows+1)
age = np.random.normal(60, 20, nRows).astype(int)

# Remove patients with age lower than 18 or higher than 100
invalid = age[np.logical_or(age < 18, age > 100)].size
while invalid > 0:
    age[np.logical_or(age < 18, age > 100)] = np.random.normal(75, 10, invalid).astype(int)
    invalid = age[np.logical_or(age < 18, age > 100)].size

# diagnosis = np.asarray([fake.diagnosis() for i in range(nRows)])
address = np.asarray([fake.street_address() for i in range(nRows)])
zip_code = np.asarray([fake.postcode() for i in range(nRows)])
city = np.asarray([fake.city_name() for i in range(nRows)])
phone_number = np.asarray([fake.unique.phone_number() for i in range(nRows)])

In [131]:
name = []
gender = []
weight = np.empty(nRows).astype(int)
height = np.empty(nRows).astype(int)

for i in range(nRows):
    rand = np.random.rand()
    if rand < 0.48:
        name.append(fake.unique.name_male())
        gender.append("Male")
        height[i] = int(np.random.normal(180, 7.5))
        weight[i] = int(np.random.normal(80, 10))
    elif rand < 0.98:
        name.append(fake.unique.name_female())
        gender.append("Female")
        height[i] = int(np.random.normal(167, 5))
        weight[i] = int(np.random.normal(60, 7.5))
    else:
        name.append(fake.unique.name_nonbinary())
        gender.append("Non-Binary")
        height[i] = int(np.random.normal(175, 6))
        weight[i] = int(np.random.normal(70, 10))

name = np.asarray(name)
gender = np.asarray(gender)

In [132]:
diagnosis = []
medication = []

for i in range(nRows):
    rand = np.random.rand()
    if rand < 0.09:
        diagnosis.append("E10")
        medication.append("Insulin")
    elif rand < 0.98:
        diagnosis.append("E11")
        medication.append("Metformin")
    else:
        diagnosis.append("E13")
        medication.append("Insulin")

diagnosis = np.asarray(diagnosis)
medication = np.asarray(medication)
glucose = np.asarray([np.random.randint(60, 450) for i in range(nRows)])
HbA1C = np.asarray([round(np.random.uniform(4, 12), 2) for i in range(nRows)])

In [133]:
ersatz_kassen = [104940005, 103306961, 104450915, 109519176, 103508742, 101002659, 101575519] 
ikks = [109888001, 108888888, 109500787, 109500044, 109500490, 109500398] 
aok =  [108918320, 108814099, 108928697, 108811072, 108815718] 
bkk = [108313123, 108918428, 108817930, 108811215, 108334056, 108815217, 108312586]  
lkk = [109000051] 
alle_kassen = ersatz_kassen + ikks + aok + bkk + lkk
kassen_gewichte = (
    [0.4 / len(ersatz_kassen)] * len(ersatz_kassen) +
    [0.07 / len(ikks)] * len(ikks) +
    [0.37 / len(aok)] * len(aok) +
    [0.15 / len(bkk)] * len(bkk) +
    [0.01]  
)
insurance_company = np.asarray([random.choices(
    alle_kassen,
    weights=kassen_gewichte,
    k=1
)[0]for i in range(nRows)])
insurance_number = np.asarray([fake.bothify(text='?#########', letters='ABCDEFGHIJKLMNOPQRSTUVWXYZ') for i in range(nRows)])

### Single Dataset

In [134]:
name[nRows - 1] = "Henri Allgöwer"
address[nRows - 1] = "Einsteinufer 17"
zip_code[nRows - 1] = 10587
phone_number[nRows - 1] = "01765 123456"
gender[nRows - 1] = "Male"
height[nRows - 1] = 178
weight[nRows - 1] = 68
age[nRows - 1] = 27
insurance_company[nRows - 1] = 101575519
diagnosis[nRows - 1] = "E10"
glucose[nRows - 1] = 453
HbA1C[nRows - 1] = 10.13
medication[nRows - 1] = "Insulin"

data = {'id': pid,
    'name': name, 
    'address': address,
    'zip': zip_code,
    'phone': phone_number,
    'gender': gender, 
    'height': height,
    'weight': weight,
    'age': age,
    'insurance_company': insurance_company,
    'insurance_number': insurance_number,
    'diagnosis': diagnosis,
    'glucose': glucose,
    'HbA1C': HbA1C,
    'medication': medication
}
data

{'id': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 17

In [135]:
df_synthetic_data = pd.DataFrame.from_dict(data=data)
#df['phone'] = df['phone'].str.replace(r'[()]',"", regex=True).str.replace(r' ',"", regex=True).str.replace("+49","", regex=False)
df_synthetic_data

,id,name,address,zip,phone,gender,height,weight,age,insurance_company,insurance_number,diagnosis,glucose,HbA1C,medication
0,1,Frank-Peter Finke,Benthinring 6/4,63634,+49(0)6922 01205,Male,193,63,95,108313123,A676434302,E10,215,11.10,Insulin
1,2,Hardy Henschel,Sauerplatz 42,27554,(05876) 360586,Male,167,82,68,108811072,D568238630,E11,303,7.54,Metformin
2,3,Paola Rudolph,Reinhardtweg 41,53272,(07996) 94991,Female,162,63,79,108888888,E131064992,E11,289,4.02,Metformin
3,4,Manfred Mitschke,Mühlestr. 565,37218,(02428) 98642,Male,180,56,78,103508742,O293579875,E11,118,6.28,Metformin
4,5,Ljudmila Schmiedecke,Martinallee 0,52152,(09146) 46907,Female,171,51,97,108918320,F524358073,E11,180,10.51,Metformin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Ivan Schaaf-Wilms,Fredo-Schuchhardt-Straße 94,90498,08848 69317,Male,173,76,58,108815217,F484319828,E11,352,9.04,Metformin
496,497,Dipl.-Ing. Theodora Jähn,Klemmplatz 5/0,79069,+49(0)4722 545949,Female,170,67,46,108811072,L739061003,E11,296,6.18,Metformin
497,498,Mikhail Hartmann,Ramona-Ditschlerin-Weg 449,00942,+49(0) 396231019,Male,178,81,49,103306961,J258936323,E11,328,11.65,Metformin
498,499,Sergei Kranz B.Eng.,Käte-Albers-Allee 6/6,04504,+49(0)1274 092621,Male,178,76,39,103306961,C790703966,E10,86,7.15,Insulin


In [136]:
dir = os.getcwd()
df_synthetic_data.to_csv(dir + "/datasets/single/syntheticData.csv", index=False)

### Multiple Dataset

In [137]:
data_patient = {
    'id' : pid,
    'name': name, 
    'diagnosis': diagnosis,
    'phone': phone_number
}

In [138]:
df = pd.DataFrame.from_dict(data=data_patient)
#df['phone'] = df['phone'].str.replace(r'[()]',"", regex=True).str.replace(r' ',"", regex=True).str.replace("+49","", regex=False)
df

,id,name,diagnosis,phone
0,1,Frank-Peter Finke,E10,+49(0)6922 01205
1,2,Hardy Henschel,E11,(05876) 360586
2,3,Paola Rudolph,E11,(07996) 94991
3,4,Manfred Mitschke,E11,(02428) 98642
4,5,Ljudmila Schmiedecke,E11,(09146) 46907
...,...,...,...,...
495,496,Ivan Schaaf-Wilms,E11,08848 69317
496,497,Dipl.-Ing. Theodora Jähn,E11,+49(0)4722 545949
497,498,Mikhail Hartmann,E11,+49(0) 396231019
498,499,Sergei Kranz B.Eng.,E10,+49(0)1274 092621


In [139]:
df.to_csv(dir + "/datasets/multiple/patient.csv", index=False)

In [140]:
data_physical = {
    'id': pid,
    'gender': gender,
    'age': age,
    'height': height,
    'weight': weight,
}

In [141]:
df = pd.DataFrame(data=data_physical)
df

,id,gender,age,height,weight
0,1,Male,95,193,63
1,2,Male,68,167,82
2,3,Female,79,162,63
3,4,Male,78,180,56
4,5,Female,97,171,51
...,...,...,...,...,...
495,496,Male,58,173,76
496,497,Female,46,170,67
497,498,Male,49,178,81
498,499,Male,39,178,76


In [142]:
df.to_csv(dir + "/datasets/multiple/physical.csv")

In [143]:
data_address = {
    'id': pid,
    'address': address,
    'zip': zip_code,
    'city' : city
}

In [144]:
df = pd.DataFrame(data=data_address)
df

,id,address,zip,city
0,1,Benthinring 6/4,63634,Tuttlingen
1,2,Sauerplatz 42,27554,Gotha
2,3,Reinhardtweg 41,53272,Eckernförde
3,4,Mühlestr. 565,37218,Saulgau
4,5,Martinallee 0,52152,Meißen
...,...,...,...,...
495,496,Fredo-Schuchhardt-Straße 94,90498,Malchin
496,497,Klemmplatz 5/0,79069,Hansestadttralsund
497,498,Ramona-Ditschlerin-Weg 449,00942,Gräfenhainichen
498,499,Käte-Albers-Allee 6/6,04504,Kyritz


In [145]:
np.unique(address).size

500

In [146]:
df.to_csv(dir + "/datasets/multiple/contact.csv")

## Generate Masked Tables

In [147]:
b_size = 5
nFields = 2
level = 1
relNoise = 0.05
nFields_phone = 3

masked_low = mask(df_synthetic_data, bucketize_age, 'age', b_size)
masked_low = mask(masked_low, blur_zip, 'zip', nFields)
masked_low = mask(masked_low, generalize_diagnosis, 'diagnosis', level)
masked_low = mask(masked_low, add_relative_noise, 'height', relNoise)
masked_low = mask(masked_low, blur_phone, 'phone', nFields_phone)

In [148]:
masked_low.to_csv(dir + "/datasets/masked/masked_low.csv")

In [149]:
b_size = 10
nFields = 3
level = 2
relNoise = 0.10
nFields_phone = 5

masked_medium = mask(df_synthetic_data, bucketize_age, 'age', b_size)
masked_medium = mask(masked_medium, blur_zip, 'zip', nFields)
masked_medium = mask(masked_medium, generalize_diagnosis, 'diagnosis', level)
masked_medium = mask(masked_medium, add_relative_noise, 'height', relNoise)
masked_medium = mask(masked_medium, blur_phone, 'phone', nFields_phone)

In [150]:
masked_medium.to_csv(dir + "/datasets/masked/masked_medium.csv")

In [151]:
b_size = 20
nFields = 4
level = 2
relNoise = 0.20
nFields_phone = 7

masked_high = mask(df_synthetic_data, bucketize_age, 'age', b_size)
masked_high = mask(masked_high, blur_zip, 'zip', nFields)
masked_high = mask(masked_high, generalize_diagnosis, 'diagnosis', level)
masked_high = mask(masked_high, add_relative_noise, 'height', relNoise)
masked_high = mask(masked_high, blur_phone, 'phone', nFields_phone)

In [152]:
masked_high.to_csv(dir + "/datasets/masked/masked_high.csv")

In [153]:
masked_high

,id,name,address,zip,phone,gender,height,weight,age,insurance_company,insurance_number,diagnosis,glucose,HbA1C,medication
0,1,Frank-Peter Finke,Benthinring 6/4,6XXXX,+49(0)692XXXXXXX,Male,221,63,[80.0 - 99.0],108313123,A676434302,XX.X,215,11.10,Insulin
1,2,Hardy Henschel,Sauerplatz 42,2XXXX,(05876)XXXXXXX,Male,177,82,[60.0 - 79.0],108811072,D568238630,XX.X,303,7.54,Metformin
2,3,Paola Rudolph,Reinhardtweg 41,5XXXX,(07996XXXXXXX,Female,179,63,[60.0 - 79.0],108888888,E131064992,XX.X,289,4.02,Metformin
3,4,Manfred Mitschke,Mühlestr. 565,3XXXX,(02428XXXXXXX,Male,165,56,[60.0 - 79.0],103508742,O293579875,XX.X,118,6.28,Metformin
4,5,Ljudmila Schmiedecke,Martinallee 0,5XXXX,(09146XXXXXXX,Female,154,51,[80.0 - 99.0],108918320,F524358073,XX.X,180,10.51,Metformin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Ivan Schaaf-Wilms,Fredo-Schuchhardt-Straße 94,9XXXX,0884XXXXXXX,Male,156,76,[40.0 - 59.0],108815217,F484319828,XX.X,352,9.04,Metformin
496,497,Dipl.-Ing. Theodora Jähn,Klemmplatz 5/0,7XXXX,+49(0)4722XXXXXXX,Female,147,67,[40.0 - 59.0],108811072,L739061003,XX.X,296,6.18,Metformin
497,498,Mikhail Hartmann,Ramona-Ditschlerin-Weg 449,0XXXX,+49(0) 39XXXXXXX,Male,196,81,[40.0 - 59.0],103306961,J258936323,XX.X,328,11.65,Metformin
498,499,Sergei Kranz B.Eng.,Käte-Albers-Allee 6/6,0XXXX,+49(0)1274XXXXXXX,Male,183,76,[20.0 - 39.0],103306961,C790703966,XX.X,86,7.15,Insulin


In [154]:
df['zip']

0      63634
1      27554
2      53272
3      37218
4      52152
       ...  
495    90498
496    79069
497    00942
498    04504
499    10587
Name: zip, Length: 500, dtype: object

In [155]:
a = 2
b = 3
(a,b)

(2, 3)